In [25]:
import re 
from pathlib import Path

import dotenv
import streamlit as st
from loguru import logger
#from streamlit_markmap import markmap

from lawsy.app.utils.lm import load_lm
from lawsy.app.utils.preload import (
    load_mindmap_maker,
    load_outline_creater,
    load_query_expander,
    load_stream_report_writer,
    load_text_encoder,
    load_vector_search_article_retriever,
)
from lawsy.reranker.rrf import RRF

In [2]:
dotenv.load_dotenv()
#css = (Path(__file__).parent.parent / "styles" / "style.css").read_text()
css = (Path.cwd() / "lawsy" / "app" / "styles" / "style.css").read_text()

#st.markdown(f"<style>{css}</style>", unsafe_allow_html=True)

text_encoder = load_text_encoder()
vector_search_article_retriever = load_vector_search_article_retriever()

gpt_4o = "openai/gpt-4o"
gpt_4o_mini = "openai/gpt-4o-mini"
# gemini_pro = "vertex_ai/gemini-2.0-exp-02-05"
# gemini_flash = "vertex_ai/gemini-2.0-flash-001"
# gemini_flash_lite = "vertex_ai/gemini-2.0-flash-lite-preview-02-05"


mindmap_maker_lm = load_lm(gpt_4o_mini)
mindmap_maker = load_mindmap_maker(_lm=mindmap_maker_lm)
rrf = RRF()

2025-02-13 00:53:03.041 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:03.042 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-02-13 00:53:03.042 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:03.192 
  command:

    streamlit run /home/hrkzz/anaconda3/envs/chat2/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-13 00:53:03.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:03.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:03.194 | INFO     | lawsy.app.utils.preload

In [3]:
query = """
国の会計制度には、調達や支出の実績に関する報告を求める法令があります。例えば、 国等による環境物品等の調達の推進等に関する法律に基づく環境物品等の調達実績の公表、官公需法に基づく契約実績額の公表などがあります。
レポートでは、これらと同じように調達、契約、支払に関する公表を求める法令を可能な限りすべて調査し、法令の概要、公表すべき内容等をなるべく詳細にまとめ、そのうち重複等無駄があるものを指摘し、今後の対応策を教えてください。
"""

In [4]:
query_expander_lm = load_lm(gpt_4o_mini)
query_expander = load_query_expander(_lm=query_expander_lm)

2025-02-13 00:53:36.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:36.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:36.832 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:36.832 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:36.836 | INFO     | lawsy.app.utils.preload:load_query_expander:73 - loading query expander...
2025-02-13 00:53:36.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:36.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:53:36.841 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-0

In [5]:
query_expander_result = query_expander(query=query)
expanded_queries = [query] + query_expander_result.topics

In [6]:
for i, topic in enumerate(query_expander_result.topics, start=1):
    print(f"[{i}] {topic}")

[1] 国等による環境物品等の調達の推進等に関する法律（環境物品調達法）に基づく調達実績の公表
[2] 官公需契約に関する法律（官公需法）に基づく契約実績額の公表
[3] 公共工事の入札及び契約の適正化に関する法律に基づく契約実績の公表
[4] 地方自治法第234条に基づく地方公共団体の契約に関する報告義務
[5] 公共調達に関する法律（公共調達法）に基づく調達実績の公表
[6] 財政法第4条に基づく支出の報告義務
[7] 国有財産法第3条に基づく国有財産の管理及び処分に関する報告
[8] 予算及び決算に関する法律に基づく予算執行状況の公表
[9] 政府調達に関する国際的な合意（WTO政府調達協定）に基づく報告義務
[10] 公共サービスの提供に関する法律に基づくサービス契約の公表義務
[11] 情報公開法に基づく調達及び契約に関する情報の公開
[12] 競争入札に関する法律に基づく入札結果の公表


In [7]:
runs = []
key2result = {}
for expanded_query in expanded_queries:
    query_vector = text_encoder.get_query_embeddings([expanded_query])[0]
    hits = vector_search_article_retriever.search(query_vector, k=5)
    run = {}
    for result in hits:
        key = (result.law_id, result.anchor)
        run[key] = result.score
        key2result[key] = result
    runs.append(run)
fused_ranks = rrf(runs)
search_results = []
for key, _ in sorted(fused_ranks.items(), key=lambda item: item[1])[::-1]:
    result = key2result[key]
    search_results.append(result)
print(f"found {len(search_results)} sources:")
for i, result in enumerate(search_results, start=1):
    print(f"[{i}] " + result.title)

found 36 sources:
[1] 公共工事の入札及び契約の適正化の促進に関する法律 附則 第五条
[2] 公共工事の入札及び契約の適正化の促進に関する法律 附則 第八条
[3] 国等による環境物品等の調達の推進等に関する法律 附則 第八条
[4] 官公需についての中小企業者の受注の確保に関する法律 附則 第六条
[5] 国等による環境物品等の調達の推進等に関する法律 附則 第七条
[6] 公共工事の入札及び契約の適正化の促進に関する法律施行令 附則 第四条
[7] 国等による障害者就労施設等からの物品等の調達の推進等に関する法律 附則 第七条
[8] 競争の導入による公共サービスの改革に関する法律施行令 附則 第七条
[9] 経済連携協定に基づく特定原産地証明書の発給等に関する法律 附則 第三十条
[10] 広域的運営推進機関の財務及び会計に関する省令 附則 第十三条
[11] 国有財産法 附則 第十条
[12] 財政法 附則 第四十六条
[13] 地方公営企業法施行令 附則 第二十八条
[14] 競争の導入による公共サービスの改革に関する法律 附則 第二十条
[15] 民間海外援助事業の推進のための物品の譲与に関する法律 附則 第三条
[16] 予算決算及び会計令 附則 第十六条
[17] 国有財産法 附則 第九条の五
[18] 会計法 附則 第四十六条
[19] 地域経済牽引事業の促進による地域の成長発展の基盤強化に関する法律 附則 第四十一条
[20] 競争の導入による公共サービスの改革に関する法律施行令 附則 第六条
[21] 公共工事の入札及び契約の適正化の促進に関する法律施行令 附則 第二条
[22] 民間資金等の活用による公共施設等の整備等の促進に関する法律施行規則 附則 第四条
[23] 経済連携協定に基づく申告原産品に係る情報の提供等に関する法律 附則 第三条
[24] 金融経済教育推進機構に関する内閣府令 附則 第二十五条
[25] 国有財産法施行令 附則 第三条
[26] 予算決算及び会計令 附則 第十八条の四
[27] 消費生活用製品安全法施行令第十四条第二項の規定に基づく都道府県知事又は市長の報告に関する省令 附則 第二条
[28] 公共工事の入札及び契約の適正化の促進に関する法律施行令 附則 第七条
[29] 国等による環境物品

In [8]:
references = []
seen = set()
for i, result in enumerate(search_results, start=1):
    if (result.rev_id, result.anchor) in seen:
        continue
    chunk_after_title = "\n".join(result.snippet.split("\n")[1:])
    references.append(f"[{i}] {result.title}\n{chunk_after_title[:1024]}")
    seen.add((result.rev_id, result.anchor))
print("\n\n".join(references)[:500])


[1] 公共工事の入札及び契約の適正化の促進に関する法律 附則 第五条
  本則
    第二章　情報の公表
      第五条
      各省各庁の長は、政令で定めるところにより、次に掲げる事項を公表しなければならない。
         一. 入札者の商号又は名称及び入札金額、落札者の商号又は名称及び落札金額、入札の参加者の資格を定めた場合における当該資格、指名競争入札における指名した者の商号又は名称その他の政令で定める公共工事の入札及び契約の過程に関する事項
         二. 契約の相手方の商号又は名称、契約金額その他の政令で定める公共工事の契約の内容に関する事項

[2] 公共工事の入札及び契約の適正化の促進に関する法律 附則 第八条
  本則
    第二章　情報の公表
      第八条
      地方公共団体の長は、政令で定めるところにより、次に掲げる事項を公表しなければならない。
         一. 入札者の商号又は名称及び入札金額、落札者の商号又は名称及び落札金額、入札の参加者の資格を定めた場合における当該資格、指名競争入札における指名した者の商号又は名


In [9]:
references[:2]

['[1] 公共工事の入札及び契約の適正化の促進に関する法律 附則 第五条\n  本則\n    第二章\u3000情報の公表\n      第五条\n      各省各庁の長は、政令で定めるところにより、次に掲げる事項を公表しなければならない。\n         一. 入札者の商号又は名称及び入札金額、落札者の商号又は名称及び落札金額、入札の参加者の資格を定めた場合における当該資格、指名競争入札における指名した者の商号又は名称その他の政令で定める公共工事の入札及び契約の過程に関する事項\n         二. 契約の相手方の商号又は名称、契約金額その他の政令で定める公共工事の契約の内容に関する事項',
 '[2] 公共工事の入札及び契約の適正化の促進に関する法律 附則 第八条\n  本則\n    第二章\u3000情報の公表\n      第八条\n      地方公共団体の長は、政令で定めるところにより、次に掲げる事項を公表しなければならない。\n         一. 入札者の商号又は名称及び入札金額、落札者の商号又は名称及び落札金額、入札の参加者の資格を定めた場合における当該資格、指名競争入札における指名した者の商号又は名称その他の政令で定める公共工事の入札及び契約の過程に関する事項\n         二. 契約の相手方の商号又は名称、契約金額その他の政令で定める公共工事の契約の内容に関する事項']

In [10]:
outline_creater_lm = load_lm(gpt_4o_mini)
outline_creater = load_outline_creater(_lm=outline_creater_lm)

2025-02-13 00:54:12.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:54:12.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:54:12.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:54:12.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:54:12.226 | INFO     | lawsy.app.utils.preload:load_outline_creater:81 - loading outline creater...
2025-02-13 00:54:12.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:54:12.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:54:12.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025

In [11]:
outline_creater_result = outline_creater(
    query=query, topics=query_expander_result.topics, references=references
)
print("generated outline:")
print(outline_creater_result.outline)


generated outline:
# 国の会計制度における調達・契約・支出に関する公表法令の調査

## 調達に関する法令
### 環境物品等の調達の推進に関する法律
[3][5][29]
### 障害者就労施設等からの物品等の調達の推進に関する法律
[7]
### 官公需についての中小企業者の受注の確保に関する法律
[4]

## 契約に関する法令
### 公共工事の入札及び契約の適正化の促進に関する法律
[1][2][6][28]
### 競争の導入による公共サービスの改革に関する法律
[14][20]
### 民間資金等の活用による公共施設等の整備等の促進に関する法律
[30]

## 支出に関する法令
### 財政法
[12][18]
### 予算決算及び会計令
[16][26]
### 国有財産法
[11][17][25]

## 公表内容の概要
### 公表すべき情報の種類
[1][3][4][12]
### 公表の頻度と方法
[6][21][33]

## 重複と無駄の指摘
### 法令間の重複
[4][7][29]
### 公表内容の冗長性
[3][12][18]

## 今後の対応策
### 法令の整理と統合
[4][7][29]
### 公表方法の改善
[6][21][30]


In [12]:
def _split_outline(outline: str):
    lines = outline.splitlines()
    overall_title = ""
    sections = []
    current_section = []
    for line in lines:
        if line.startswith("# ") and not line.startswith("##"):
            overall_title = line.strip()
        elif line.startswith("## "):
            if current_section:
                sections.append("\n".join(current_section))
                current_section = []
            current_section.append(line.strip())
        elif current_section:
            current_section.append(line.strip())
    if current_section:
        sections.append("\n".join(current_section))
    return overall_title, sections
    
overall_title, sections = _split_outline(outline_creater_result.outline)

In [15]:
def _parse_references(references: list[str]) -> dict[int, str]:
    """リファレンスリストを {番号: 内容} の辞書に変換"""
    references_dict = {}
    for ref in references:
        match = re.match(r"\[(\d+)\]\s*(.+)", ref, re.DOTALL)
        if match:
            ref_num = int(match.group(1))  # 例: "[1]" → 1
            ref_content = match.group(2)   # 例: "公共工事の入札..."
            references_dict[ref_num] = f"[{ref_num}] {match.group(2)}"
    return references_dict

references_dict = _parse_references(references=references)

In [16]:
def _extract_references_from_outline(section_outline: str) -> list[int]:
    """セクションのアウトラインから引用番号を抽出"""
    reference_ids = list(map(int, re.findall(r'\[(\d+)\]', section_outline)))
    return reference_ids

In [17]:
for idx, sec_outline in enumerate(sections):
    reference_ids = _extract_references_from_outline(sec_outline)
    print(reference_ids)
    filtered_references = "\n\n".join([references_dict[rid] for rid in reference_ids if rid in references_dict])
    print(filtered_references)

[3, 5, 29, 7, 4]
[3] 国等による環境物品等の調達の推進等に関する法律 附則 第八条
  本則
    第八条 （調達実績の概要の公表等）
    各省各庁の長及び独立行政法人等の長は、毎会計年度又は毎事業年度の終了後、遅滞なく、環境物品等の調達の実績の概要を取りまとめ、公表するとともに、環境大臣に通知するものとする。
    前項の規定による環境大臣への通知は、独立行政法人等の長にあっては、当該独立行政法人等の主務大臣を通じて行うものとする。

[5] 国等による環境物品等の調達の推進等に関する法律 附則 第七条
  本則
    第七条 （環境物品等の調達方針）
    各省各庁の長及び独立行政法人等の長（当該独立行政法人等が特殊法人である場合にあっては、その代表者。以下同じ。）は、毎年度、基本方針に即して、物品等の調達に関し、当該年度の予算及び事務又は事業の予定等を勘案して、環境物品等の調達の推進を図るための方針を作成しなければならない。
    前項の方針は、次に掲げる事項について定めるものとする。
       一. 特定調達物品等の当該年度における調達の目標
       二. 特定調達物品等以外の当該年度に調達を推進する環境物品等及びその調達の目標
       三. その他環境物品等の調達の推進に関する事項
    各省各庁の長及び独立行政法人等の長は、第一項の方針を作成したときは、遅滞なく、これを公表しなければならない。
    各省各庁の長及び独立行政法人等の長は、第一項の方針に基づき、当該年度における物品等の調達を行うものとする。

[29] 国等による環境物品等の調達の推進等に関する法律 附則 第六条
  本則
    第六条 （環境物品等の調達の基本方針）
    国は、国及び独立行政法人等における環境物品等の調達を総合的かつ計画的に推進するため、環境物品等の調達の推進に関する基本方針（以下「基本方針」という。）を定めなければならない。
    基本方針は、次に掲げる事項について定めるものとする。
       一. 国及び独立行政法人等による環境物品等の調達の推進に関する基本的方向
       二. 国及び独立行政法人等が重点的に調達を推進すべき環境物品等の種類（以下「特定調達品目」という。）及びその判断の基準並びに当

In [26]:
report_writer_lm = load_lm(gpt_4o_mini)
stream_report_writer = load_stream_report_writer(_lm=report_writer_lm)
report_stream = stream_report_writer(
    query=query, outline=outline_creater_result.outline, references=references
)


2025-02-13 00:59:03.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 00:59:03.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
for chunk in report_stream:
    print(chunk)

# 国の会計制度における調達・契約・支出に関する公表法令の調査

本レポートでは、国の会計制度における調達、契約、支出に関する公表を求める法令を包括的に調査し、それぞれの法令の概要や公表すべき内容を詳細にまとめます。特に、環境物品等の調達の推進に関する法律や官公需法など、既存の法令間に見られる重複や無駄を指摘し、今後の法令整理や公表方法の改善に向けた具体的な対応策を提案します。これにより、透明性の向上と効率的な会計制度の実現を目指します。



KeyboardInterrupt: 

: 

In [22]:
full_report = stream_report_writer.get_text()

In [23]:
import markdown
import re
from IPython.display import FileLink

# 1. Markdown 文字列を取得
markdown_text = full_report  # メインのMarkdownテキスト
references_text = "\n\n".join(references)  # 参考文献を結合
full_markdown = f"{markdown_text}\n\n## 参考文献\n\n{references_text}"  # すべて結合

# 2. タイトルをMarkdownのH1（#）から抽出
match = re.search(r'^#\s+(.*)', markdown_text, re.MULTILINE)
title = match.group(1) if match else "レポート"

# 3. 参考文献の改行を `<br>` タグでラップする
formatted_references = "<br><br>".join([ref.replace("\n", "<br>") for ref in references])

# 4. Markdown を HTML に変換（本文は Markdown 変換、参考文献は直接 HTML）
html_body = markdown.markdown(markdown_text, extensions=['extra'])
html_references = f"<h2>参考文献</h2>\n{formatted_references}"

# 5. HTML テンプレートを作成（タイトルを動的に設定）
html_template = f"""
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{title}</title>
    <style>
        body {{ font-family: Arial, sans-serif; line-height: 1.6; padding: 20px; }}
        h1 {{ font-size: 32px; color: black; }}
        h2 {{ font-size: 26px; color: black; margin-top: 20px; margin-bottom: 5px; border-top: 0.5px solid black; }}
        h3 {{ font-size: 22px; color: black; margin-top: 10px; margin-bottom: 5px; }}
        p, li {{ font-size: 16px; line-height: 1.8; margin-top: 10px; margin-bottom: 5px;  }}
        ul {{ padding-left: 20px; }}
        li {{ margin-bottom: 10px; }}
        pre {{ background: #f4f4f4; padding: 10px; border-radius: 5px; }}
    </style>
</head>
<body>
    {html_body}
    {html_references}
</body>
</html>
"""

# 6. HTML ファイルとして保存
file_path = "report.html"
with open(file_path, "w", encoding="utf-8") as f:
    f.write(html_template)

# 7. ダウンロードリンクを表示
display(FileLink(file_path))


/home/hrkzz/law-dx-hackathon-2025/src/report.html

In [17]:
import os
import concurrent.futures
import dspy

class ExpandRefrences(dspy.Signature):
    """
    あなたは日本の法令に精通した専門家です。下記のクエリに関する情報としてreferencesにある関連条文が収集されました。referencesの中で引用されている条文を収集すべく、再度ベクトル検索を実施したい。referencesの中で引用されている条文をすべて抽出してください。
    手順は以下。
    1. 「前項に規定する」「前号に定める」「この章において定める」等：
       - 収集された条文のタイトルや条文番号を参照し、どの法令のどの条文を指しているかを推測し、条文番号を生成

    2. 「*法第*条に規定する」「*法第*条*項に規定する」「*法第*項に基づく」等：
       - 記載されている法令の具体的な条文番号を抽出
       
    3. 政令、省令、規則等の下位法令：
       - どういった上位法令のどの条文を指しているかを推測し、条文番号を生成
    
    クエリーの回答に必要でこれまで収集できていなかったものについて、条文番号まで特定した検索文に変換してください。
    最後に重複を排除して、最低限の検索文に集約してください。
    
    出力フォーマット：
    - topic 1: 
    - ...
    - topic n: 
    ...  
    """

    query = dspy.InputField(desc="クエリー", format=str)
    references = dspy.InputField(desc="収集された情報源と引用番号", format=str)
    expanded_topics = dspy.OutputField(desc="レポートのアウトライン", format=str)


class RefrencesExpander(dspy.Module):
    def __init__(self, lm) -> None:
        self.lm = lm
        self.expand_references = dspy.Predict(ExpandRefrences)
    
    def forward(self, query: str, references: list[str]) -> dspy.Prediction:
        # 並列処理のための最大スレッド数を設定
        max_thread_num: int = min(20, (os.cpu_count() or 1) * 2)
        expanded_topics_list = []

        # dspyのコンテキスト内で処理する
        with dspy.settings.context(lm=self.lm):
            # ThreadPoolExecutorを用いて各リファレンスを並列処理する
            with concurrent.futures.ThreadPoolExecutor(max_workers=max_thread_num) as executor:
                # 各リファレンスを個別に submit する
                future_to_ref = {
                    executor.submit(self.expand_references, query=query, references=ref): ref
                    for ref in references
                }
                # 並列処理結果を取得する
                for future in concurrent.futures.as_completed(future_to_ref):
                    try:
                        result = future.result()
                        expanded_topics_list.append(result.expanded_topics)
                    except Exception as exc:
                        print(f"リファレンスの処理中に例外が発生しました: {exc}")

        # 各結果を統合して最終的なアウトプットとする
        combined_topics = "\n\n".join(expanded_topics_list)
        return dspy.Prediction(expanded_topics=combined_topics)


In [18]:
references_expander = RefrencesExpander(lm=gpt_4o_mini)
references_expander_result = references_expander(query=query, references=references)

In [19]:
print(references_expander_result.expanded_topics)

- 消防法 第2条: 火災の予防に関する基本的な規定
- 消防法 第8条の2の2: 防火対象物の点検に関する規定
- 消防法 第8条第1項: 防火対象物の管理に関する権限と義務
- 消防法 第17条の3の3: 特定の点検及び報告の対象となる事項
- 総務省令: 防火対象物点検資格者の資格及び点検基準に関する詳細規定

- 医薬品、医療機器等の品質、有効性及び安全性の確保等に関する法律 第15条
- 医療機器及び体外診断用医薬品の製造販売業者等の遵守事項等に関する規定
- 医療機器の製造管理及び品質管理に関する省令
- 医療機器責任技術者及び体外診断用医薬品製造管理者の義務に関する規定
- 医療機器の試験検査の実施方法に関する規定
- 製造販売後安全管理に係る業務の委託に関する規定

- 職業能力開発促進法第15条の7第1項に基づく職業訓練の要件
- 職業能力開発促進法施行規則第2条における職業訓練の実施方法
- 目視規制に関連する法令の特定
- 実地監査規制に関連する法令の特定
- 定期検査・点検規制に関連する法令の特定
- 常駐・専任規制に関連する法令の特定
- 対面講習規制に関連する法令の特定
- 書面掲示規制に関連する法令の特定
- 往訪閲覧・縦覧規制に関連する法令の特定

- 目視規制に関する条文: 法第六条第六項に基づく主務省令の具体的な内容
- 実地監査規制に関する条文: 行政機関が書面等の原本確認を求める場合の規定
- 定期検査・点検規制に関する条文: 定期的な検査や点検を求める法令の特定
- 常駐・専任規制に関する条文: 特定の者に対する常時滞在義務に関する規定
- 対面講習規制に関する条文: 国家資格等の講習に関する対面での実施義務
- 書面掲示規制に関する条文: 公的証明書等の掲示に関する規定
- 往訪閲覧・縦覧規制に関する条文: 公的情報の閲覧に訪問が必要な場合の規定

- 目視規制に関する条文: 行政手続における特定の個人を識別するための番号の利用等に関する法律 第35条
- 実地監査規制に関する条文: 行政手続における特定の個人を識別するための番号の利用等に関する法律 第35条
- 定期検査・点検規制に関する条文: 行政手続における特定の個人を識別するための番号の利用等に関する法律 第35条
- 常駐・専任規制に関する条文: 具体的な条文は

In [20]:
old_references_text = references_expander_result.expanded_topics  # 文字列
if old_references_text.strip():
    old_references = old_references_text.split("\n\n")
else:
    old_references = []
    
from tqdm import tqdm

runs = []
key2point = {}
for expanded_query in tqdm(old_references):
    query_vector = text_encoder.get_query_embeddings([expanded_query])[0]
    hits = vector_search_retriever.search(query_vector, k=2)
    run = {}
    for point in hits:
        file_name = point.meta["file_name"]  # type: ignore
        law_id = file_name.split("_")[0]
        anchor = point.meta["anchor"]  # type: ignore
        key = (law_id, anchor)
        run[key] = point.score
        key2point[key] = point
    runs.append(run)
fused_ranks = rrf(runs)

search_result = []
for key, score in sorted(fused_ranks.items(), key=lambda item: item[1])[::-1]:
    point = key2point[key]
    new_point = Point(index=point.index, score=score, meta=point.meta)
    search_result.append(new_point)

references_add = []
seen = set()

# 既存の references の最後の番号を取得
last_index = int(references[-1].split("]")[0][1:])  # 最後の参照番号を取得

for i, point in enumerate(search_result, start=last_index + 1):
    file_name = point.meta["file_name"]  # type: ignore
    anchor = point.meta["anchor"]  # type: ignore
    law_id = file_name.split("_")[0]
    egov_url = f"https://laws.e-gov.go.jp/law/{law_id}#{anchor}"
    chunk_dict = chunks[file_name, anchor]

    if (file_name, anchor) in seen:
        continue

    article_title = get_article_title(chunk_dict)
    chunk_after_title = "\n".join(chunk_dict["chunk"].split("\n")[1:])
    
    reference_text = f"[{i}] {article_title}\n{chunk_after_title[:1024]}"
    references_add.append(reference_text)
    seen.add((file_name, anchor))

    print(f"[{i}] " + article_title)


100%|██████████| 30/30 [00:28<00:00,  1.07it/s]

[31] 特定工場における公害防止組織の整備に関する法律 第11条
[32] 建築物における衛生的環境の確保に関する法律 第11条
[33] 核原料物質、核燃料物質及び原子炉の規制に関する法律 第2条
[34] 電子委任状の普及の促進に関する法律 第13条
[35] 民間資金等の活用による公共施設等の整備等の促進に関する法律 第63条
[36] 電気通信事業法 第166条
[37] 化学物質の審査及び製造等の規制に関する法律 第2条
[38] 合法伐採木材等の流通及び利用の促進に関する法律 第40条
[39] 鉄道事業法 第56条
[40] デジタル庁の所管する法令に係る情報通信技術を活用した行政の推進等に関する法律施行規則 第7条
[41] 建築物における衛生的環境の確保に関する法律 第15条
[42] 労働安全衛生規則 第87条
[43] 職業能力開発促進法施行規則 第15条
[44] 特定家庭用機器再商品化法施行令 第6条
[45] 建築士法 第2条
[46] 特定化学物質の環境への排出量の把握等及び管理の改善の促進に関する法律 第1条
[47] 消防法施行規則 第18条
[48] 医療分野の研究開発に資するための匿名加工医療情報及び仮名加工医療情報に関する法律 第59条
[49] 労働安全衛生法 第3条
[50] 建築物における衛生的環境の確保に関する法律施行規則 第21条
[51] 行政手続における特定の個人を識別するための番号の利用等に関する法律 第35条
[52] 職業能力開発促進法施行規則 第2条
[53] 医薬品、医療機器等の品質、有効性及び安全性の確保等に関する法律 第2条
[54] 消防法 第2条
[55] 電気用品安全法 第46条
[56] デジタル庁の所管する法令に係る情報通信技術を活用した行政の推進等に関する法律施行規則 第12条
[57] 電気事業法 第107条
[58] 特定化学物質の環境への排出量の把握等及び管理の改善の促進に関する法律 第3条
[59] 令和九年に開催される国際園芸博覧会の準備及び運営のために必要な特別措置に関する法律 第8条


In [21]:
references_new = references+references_add

In [22]:
print("\n\n".join(references_new)[:500])

[1] 医薬品、医療機器等の品質、有効性及び安全性の確保等に関する法律 第15条
  本則
    第五章　医療機器及び体外診断用医薬品の製造販売業及び製造業等
      第一節　医療機器及び体外診断用医薬品の製造販売業及び製造業
        第二十三条の二の十五 （医療機器及び体外診断用医薬品の製造販売業者等の遵守事項等）
        厚生労働大臣は、厚生労働省令で、医療機器又は体外診断用医薬品の製造管理若しくは品質管理又は製造販売後安全管理の実施方法、医療機器等総括製造販売責任者の義務の遂行のための配慮事項その他医療機器又は体外診断用医薬品の製造販売業者がその業務に関し遵守すべき事項を定めることができる。
        医療機器又は体外診断用医薬品の製造販売業者は、前条第三項の規定により述べられた医療機器等総括製造販売責任者の意見を尊重するとともに、法令遵守のために措置を講ずる必要があるときは、当該措置を講じ、かつ、講じた措置の内容（措置を講じない場合にあつては、その旨及びその理由）を記録し、これを適切に保存しなければならない。
        厚生労働大臣は、厚生労働


In [17]:
outline_creater = OutlineCreater(lm=gpt_4o_mini)
outline_result = outline_creater.forward(
    query=query,
    topics=query_expander_result.topics,
    references=references,
)

In [18]:
print(outline_result.outline)

# アナログ規制に関するレポート

## 目視規制
### 目視による検査・点検
[1][3][7]
### 目視による調査
[4][9][14]
### 目視による巡視・見張り
[6][18]

## 実地監査規制
### 書類確認による判定
[2][10][19]
### 現場確認の必要性
[5][12][20]

## 定期検査・点検規制
### 第三者検査の実施
[8][11][25]
### 自主検査の義務
[17][26][29]

## 常駐・専任規制
### 常駐義務の内容
[13][21][22]
### 専任者の配置
[15][23][30]

## 対面講習規制
### 対面講習の必要性
[16][24][28]
### オンライン講習との違い
[31][32][33]

## 書面掲示規制
### 書面掲示の義務
[27][34]
### 対面確認の重要性
[1][3][4]

## 往訪閲覧・縦覧規制
### 公的機関への訪問の必要性
[2][5][6]
### 縦覧の手続き
[9][10][11]


In [21]:
parallel_report_writer = ParallelReportWriter(lm=gpt_4o_mini)
parallel_report_writer_result = parallel_report_writer(query=query, outline=outline_result.outline, references=references)


In [26]:
print(parallel_report_writer_result.report)

# アナログ規制に関する日本の法令の分析
日本の法令におけるアナログ規制は、デジタル化の進展を妨げる重要な要素として注目されています。これらの規制は、目視規制、実地監査規制、定期検査・点検規制、常駐・専任規制、対面講習規制、書面掲示規制、往訪閲覧・縦覧規制といった具体的な形で存在し、法令遵守や安全管理を目的としています。しかし、これらの規制は、デジタル技術の活用を制限し、業務の効率化を妨げる要因ともなり得ます。

目視規制は、現場での目視による確認を求めるものであり、特に医療機器や消防法において重要な役割を果たしています。実地監査規制は、書類や設備の確認を通じて法令遵守を確保するもので、特定の業種においては不可欠な手段です。定期検査・点検規制は、特定の施設や設備が法令に適合しているかを定期的に確認することを義務付けており、公共の安全や健康を守るために重要です。

さらに、常駐・専任規制は、特定の業務に専念することを求め、専門的な知識や技術が必要な分野において重要な役割を果たします。対面講習規制は、国家資格や専門的な技術を習得するために必要なものであり、受講者同士の交流を促進する効果もあります。書面掲示規制は、特定の情報を公的な場に掲示することを義務付け、透明性を確保するための重要な手段です。最後に、往訪閲覧・縦覧規制は、公的情報を閲覧する際に直接訪問を求めるもので、情報の正確性や信頼性を確保するために重要です。

これらのアナログ規制は、法令遵守を確保するために必要な側面もありますが、デジタル化の進展に対して障壁となる要因でもあります。今後は、これらの規制の見直しやデジタル化の促進が求められ、オンラインでの講習や書面掲示のデジタル化、情報の電子的な閲覧方法の導入が必要です。法令の適用範囲を見直し、デジタル技術を活用した新たな監査や点検の方法を模索することが重要です。関係機関や業界団体と連携し、アナログ規制のデジタル化に向けた具体的な提案を行うことで、法令遵守を維持しつつ、効率的な業務運営を実現することが期待されます。
## 目視規制

### 目視による検査の必要性
目視による検査は、法令が求める基準に適合しているかどうかを確認するために重要な手段です。特に、医療機器や体外診断用医薬品の製造販売業者においては、厚生労働大臣が定める基準に従い、製造管理や品質管理を適切

In [ ]:
import markdown
import re
from IPython.display import FileLink

# 1. Markdown 文字列を取得
markdown_text = parallel_report_writer_result.report  # メインのMarkdownテキスト
references_text = "\n\n".join(references)  # 参考文献を結合
full_markdown = f"{markdown_text}\n\n## 参考文献\n\n{references_text}"  # すべて結合

# 2. タイトルをMarkdownのH1（#）から抽出
match = re.search(r'^#\s+(.*)', markdown_text, re.MULTILINE)
title = match.group(1) if match else "レポート"

# 3. 参考文献の改行を `<br>` タグでラップする
formatted_references = "<br><br>".join([ref.replace("\n", "<br>") for ref in references])

# 4. Markdown を HTML に変換（本文は Markdown 変換、参考文献は直接 HTML）
html_body = markdown.markdown(markdown_text, extensions=['extra'])
html_references = f"<h2>参考文献</h2>\n{formatted_references}"

# 5. HTML テンプレートを作成（タイトルを動的に設定）
html_template = f"""
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{title}</title>
    <style>
        body {{ font-family: Arial, sans-serif; line-height: 1.6; padding: 20px; }}
        h1 {{ font-size: 32px; color: #003f66; }}
        h2 {{ font-size: 26px; color: #003f66; }}
        h3 {{ font-size: 22px; color: #003f66; }}
        p, li {{ font-size: 16px; line-height: 1.8; }}
        ul {{ padding-left: 20px; }}
        li {{ margin-bottom: 10px; }}
        pre {{ background: #f4f4f4; padding: 10px; border-radius: 5px; }}
    </style>
</head>
<body>
    {html_body}
    {html_references}
</body>
</html>
"""

# 6. HTML ファイルとして保存
file_path = "report.html"
with open(file_path, "w", encoding="utf-8") as f:
    f.write(html_template)

# 7. ダウンロードリンクを表示
display(FileLink(file_path))


/home/hrkzz/law-dx-hackathon-2025/src/lawsy/report.html

: 

In [1]:
import concurrent.futures
import dspy
import os

class WriteSection(dspy.Signature):
    """あなたは日本の法令に精通し、分かりやすい解説を書くことに定評のある信頼できるライターです。下記のクエリーに関する調査をしており、クエリーをもとにレポートのアウトラインを作成しました。
    アウトラインの中にある引用番号をもとに、収集された情報源の条文を適切に参照しながら各セクションの内容を記載してください。必ず各サブセクションごとに400字以上記載してください。解説は緻密かつ包括的で情報量が多く、情報源に基づいたものであることが望ましいです。法令に詳しくない人向けにわかりやすくかみ砕いて説明することも重要です。必要に応じて、用いている法令の概要、関連法規、適切な事例、歴史的背景、最新の判例などを盛り込んでください。
    なお、内容の信頼性が重要なので、必ず情報源にあたり、下記指示にあるように引用をするのを忘れないで下さい。
    1. アウトラインとして入力された"# Title"のレポート全体のタイトル、"## Title"のセクションのタイトル、"### Title"のサブセクションのタイトルは変更しないでください。
    2. 必ず情報源の情報に基づき記載し、ハルシネーションに気をつけること。記載の根拠となりえる参照すべき情報源は "...です[4][1][27]。" "...ます[21][9]。" のように明示してください。その記述に対しての関連性が高そうな順に付与してください。
    3. 正しく引用を明示されているほどあなたの解説は高く評価されます。引用なしの創作は論拠が明確でない限り全く評価されません。
    4. 情報源を解説の末尾に含める必要はありません。
    5. 日本語のですます調で解説を書いてください。
    """
    query = dspy.InputField(desc="クエリー", format=str)
    references = dspy.InputField(desc="収集された情報源と引用番号", format=str)
    section_title = dspy.InputField(desc="セクションのタイトルとアウトライン", format=str)
    section = dspy.OutputField(desc="生成されたセクション", format=str)

class WriteConclusion(dspy.Signature):
    """あなたは日本の法令に精通し、分かりやすい解説を書くことに定評のある信頼できるライターです。統合した各セクションを踏まえて、レポート全体の結論と今後の方向性やネクストアクションを生成します。そして、生成したconclusionの冒頭に"## 結論"という行を追記してください。
    """
    query = dspy.InputField(desc="クエリー", format=str)
    report_sections = dspy.InputField(desc="統合した各セクションの文章", format=str)
    conclusion = dspy.OutputField(desc="生成された結論", format=str)

class WriteLead(dspy.Signature):
    """あなたは日本の法令に精通し、分かりやすい解説を書くことに定評のある信頼できるライターです。統合した最終レポート全体を踏まえて、レポートのリード文を生成します。最低でも400字以上、可能なら600字以上記載し、ドラフトに追記してください。なお、リードセクションは記事の簡潔な概要として独立して成り立つものにすること（テーマを明確にし、背景を説明し、その話題がなぜ重要なのかを提示、最も重要なポイントや主要な論争があれば要約）
    """
    query = dspy.InputField(desc="クエリー", format=str)
    report_draft = dspy.InputField(desc="統合したレポート本文（結論含む）", format=str)
    lead = dspy.OutputField(desc="生成されたリード文", format=str)

class ParallelReportWriter(dspy.Module):
    def __init__(self, lm, max_thread_num: int = min(20, os.cpu_count() * 2)) -> None:
        self.lm = lm
        self.max_thread_num = max_thread_num
        self.write_section = dspy.Predict(WriteSection)
        self.write_conclusion = dspy.Predict(WriteConclusion)
        self.write_lead = dspy.Predict(WriteLead)

    def _split_outline(self, outline: str):
        lines = outline.splitlines()
        overall_title = ""
        sections = []
        current_section = []
        for line in lines:
            if line.startswith("# ") and not line.startswith("##"):
                overall_title = line.strip()
            elif line.startswith("## "):
                if current_section:
                    sections.append("\n".join(current_section))
                    current_section = []
                current_section.append(line.strip())
            elif current_section:
                current_section.append(line.strip())
        if current_section:
            sections.append("\n".join(current_section))
        return overall_title, sections

    def _generate_section(self, query: str, references_text: str, section_title: str) -> str:
        with dspy.settings.context(lm=self.lm):
            result = self.write_section(
                query=query,
                references=references_text,
                section_title=section_title
            )
        return result.section

    def forward(self, query: str, outline: str, references: list[str]) -> dspy.Prediction:
        references_text = "\n\n".join(references)
        overall_title, section_titles = self._split_outline(outline)

        section_results = {}
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_thread_num) as executor:
            future_to_idx = {
                executor.submit(self._generate_section, query, references_text, sec_title): idx
                for idx, sec_title in enumerate(section_titles)
            }
            for future in concurrent.futures.as_completed(future_to_idx):
                idx = future_to_idx[future]
                try:
                    section_results[idx] = future.result()
                except Exception as exc:
                    section_results[idx] = f"Error in section {idx}: {exc}"

        report_sections = ""
        for idx in sorted(section_results.keys()):
            report_sections += section_results[idx] + "\n"

        with dspy.settings.context(lm=self.lm):
            concl_result = self.write_conclusion(
                query=query,
                report_sections=report_sections
            )
        conclusion_text = concl_result.conclusion

        report_draft = report_sections + "\n" + conclusion_text

        with dspy.settings.context(lm=self.lm):
            lead_result = self.write_lead(
                query=query,
                report_draft=report_draft
            )
        lead_text = lead_result.lead

        final_report = overall_title + "\n" + lead_text + "\n" + report_draft

        return dspy.Prediction(report=final_report)


/home/hrkzz/anaconda3/envs/chat2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
parallel_report_writer = ParallelReportWriter(lm=gpt_4o_mini)
parallel_report_writer_result = parallel_report_writer(query=query, outline=outline_result.outline, references=references)


/tmp/ipykernel_78460/2064577141.py:2: RuntimeWarning: coroutine 'ParallelReportWriter.forward' was never awaited
  parallel_report_writer_result = parallel_report_writer(query=query, outline=outline_result.outline, references=references)


In [59]:
print(parallel_report_writer_result.report)

# 日本におけるSMR（小型モジュール炉）建設に関する法令の論点
日本における小型モジュール炉（SMR）の建設は、エネルギー政策の転換点として注目されています。特に、原子力発電が非化石エネルギーとしての役割を果たす中で、SMRはその安全性やコスト効率の面から、持続可能なエネルギー供給の実現に寄与する可能性を秘めています。しかし、SMRの建設には多くの法令が関与しており、それぞれが重要な役割を果たしています。具体的には、原子炉の設置許可をはじめ、核燃料物質の管理、環境影響評価、電気事業法、建築基準法などが挙げられます。

これらの法令は、事業の安全性を確保し、地域社会との信頼関係を築くために不可欠です。特に、原子力規制法に基づく厳格な許可制度は、発電用原子炉の設置に際して、地域住民や環境への影響を最小限に抑えるための重要な手続きとなります。また、環境影響評価法に基づく評価は、事業が環境に与える影響を事前に把握し、適切な対策を講じるための基盤となります。

さらに、電気事業法や建築基準法も、SMRの建設においては重要な要素です。これらの法令は、発電事業者が遵守すべき技術基準や安全基準を定めており、公共の安全を確保するために必要不可欠です。特に、建築基準法は、原子力関連施設の安全性や耐震性を確保するための基準を定めており、自然災害や事故に対する備えが求められます。

今後の方向性としては、これらの法令を遵守しつつ、地域住民とのコミュニケーションを強化し、透明性のあるプロセスを確立することが求められます。具体的には、法令に基づく許可申請の準備を進め、環境影響評価を実施し、地域住民の意見を反映させた計画を策定することが重要です。これにより、持続可能なエネルギー供給の実現に向けた一歩を踏み出すことができるでしょう。
## 原子力規制に関する法令

### 原子炉の設置許可
日本における小型モジュール炉（SMR）の建設には、原子炉の設置に関する厳格な許可制度が存在します。具体的には、核原料物質、核燃料物質及び原子炉の規制に関する法律（以下「原子炉等規制法」といいます）第23条に基づき、発電用原子炉を設置しようとする者は、原子力規制委員会の許可を受ける必要があります。この許可申請には、設置する原子炉の型式、熱出力、設置場所、工事計画、使用する核燃料物質の種類及び年間予定使用量など、

In [44]:
import markdown
import re
from IPython.display import FileLink

# 1. Markdown 文字列を取得
markdown_text = parallel_report_writer_result.report  # メインのMarkdownテキスト
references_text = "\n\n".join(references)  # 参考文献を結合
full_markdown = f"{markdown_text}\n\n## 参考文献\n\n{references_text}"  # すべて結合

# 2. タイトルをMarkdownのH1（#）から抽出
match = re.search(r'^#\s+(.*)', markdown_text, re.MULTILINE)
title = match.group(1) if match else "レポート"

# 3. 参考文献の改行を `<br>` タグでラップする
formatted_references = "<br><br>".join([ref.replace("\n", "<br>") for ref in references])

# 4. Markdown を HTML に変換（本文は Markdown 変換、参考文献は直接 HTML）
html_body = markdown.markdown(markdown_text, extensions=['extra'])
html_references = f"<h2>参考文献</h2>\n{formatted_references}"

# 5. HTML テンプレートを作成（タイトルを動的に設定）
html_template = f"""
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{title}</title>
    <style>
        body {{ font-family: Arial, sans-serif; line-height: 1.6; padding: 20px; }}
        h1 {{ font-size: 32px; color: #003f66; }}
        h2 {{ font-size: 26px; color: #003f66; }}
        h3 {{ font-size: 22px; color: #003f66; }}
        p, li {{ font-size: 16px; line-height: 1.8; }}
        ul {{ padding-left: 20px; }}
        li {{ margin-bottom: 10px; }}
        pre {{ background: #f4f4f4; padding: 10px; border-radius: 5px; }}
    </style>
</head>
<body>
    {html_body}
    {html_references}
</body>
</html>
"""

# 6. HTML ファイルとして保存
file_path = "report.html"
with open(file_path, "w", encoding="utf-8") as f:
    f.write(html_template)

# 7. ダウンロードリンクを表示
display(FileLink(file_path))


/home/hrkzz/law-dx-hackathon-2025/src/lawsy/report.html

In [ ]:
import dspy


class ExpandRefrences(dspy.Signature):
    """あなたは日本の法令に精通した専門家です。下記のクエリに関する情報としてreferencesにある関連条文が収集されました。referencesの中で引用されている条文を収集すべく、再度ベクトル検索を実施したい。referencesの中で引用されている条文をすべて抽出してください。
    手順は以下。
    1. 「前項に規定する」「前号に定める」「この章において定める」等：
       - 収集された条文のタイトルや条文番号を参照し、どの法令のどの条文を指しているかを推測し、条文番号を生成

    2. 「*法第*条に規定する」「*法第*条*項に規定する」「*法第*項に基づく」等：
       - 記載されている法令の具体的な条文番号を抽出
       
    3. 政令、省令、規則等の下位法令：
       - どういった上位法令のどの条文を指しているかを推測し、条文番号を生成
    
    クエリーの回答に必要でこれまで収集できていなかったものについて、条文番号まで特定した検索文に変換してください。

    出力フォーマット：
    - topic 1: 
    - ...
    - topic n: 
    ...  

    丁寧に条文を審査することはあなたの評価に繋がります。最後まで頑張ってください。
    """

    query = dspy.InputField(desc="クエリー", format=str)
    references = dspy.InputField(desc="収集された情報源と引用番号", format=str)
    expanded_topics = dspy.OutputField(desc="レポートのアウトライン", format=str)


class RefrencesExpander(dspy.Module):
    def __init__(self, lm) -> None:
        self.lm = lm
        self.expand_references = dspy.Predict(ExpandRefrences)
    
    def forward(self, query: str, references: list[str]) -> dspy.Prediction:
        references_text = "\n\n".join(references)
        with dspy.settings.context(lm=self.lm):
            expand_references_result = self.expand_references(
                query=query,
                references=references_text,
            )
        return expand_references_result


In [ ]:
import os
import concurrent.futures
import dspy

class ExpandRefrences(dspy.Signature):
    """
    あなたは日本の法令に精通した専門家です。下記のクエリに関する情報としてreferencesにある関連条文が収集されました。referencesの中で引用されている条文を収集すべく、再度ベクトル検索を実施したい。referencesの中で引用されている条文をすべて抽出してください。
    手順は以下。
    1. 「前項に規定する」「前号に定める」「この章において定める」等：
       - 収集された条文のタイトルや条文番号を参照し、どの法令のどの条文を指しているかを推測し、条文番号を生成

    2. 「*法第*条に規定する」「*法第*条*項に規定する」「*法第*項に基づく」等：
       - 記載されている法令の具体的な条文番号を抽出
       
    3. 政令、省令、規則等の下位法令：
       - どういった上位法令のどの条文を指しているかを推測し、条文番号を生成
    
    クエリーの回答に必要でこれまで収集できていなかったものについて、条文番号まで特定した検索文に変換してください。
    最後に重複を排除して、最低限の検索文に集約してください。
    
    出力フォーマット：
    - topic 1: 
    - ...
    - topic n: 
    ...  
    """

    query = dspy.InputField(desc="クエリー", format=str)
    references = dspy.InputField(desc="収集された情報源と引用番号", format=str)
    expanded_topics = dspy.OutputField(desc="レポートのアウトライン", format=str)


class RefrencesExpander(dspy.Module):
    def __init__(self, lm) -> None:
        self.lm = lm
        self.expand_references = dspy.Predict(ExpandRefrences)
    
    def forward(self, query: str, references: list[str]) -> dspy.Prediction:
        # 並列処理のための最大スレッド数を設定
        max_thread_num: int = min(20, (os.cpu_count() or 1) * 2)
        expanded_topics_list = []

        # dspyのコンテキスト内で処理する
        with dspy.settings.context(lm=self.lm):
            # ThreadPoolExecutorを用いて各リファレンスを並列処理する
            with concurrent.futures.ThreadPoolExecutor(max_workers=max_thread_num) as executor:
                # 各リファレンスを個別に submit する
                future_to_ref = {
                    executor.submit(self.expand_references, query=query, references=ref): ref
                    for ref in references
                }
                # 並列処理結果を取得する
                for future in concurrent.futures.as_completed(future_to_ref):
                    try:
                        result = future.result()
                        expanded_topics_list.append(result.expanded_topics)
                    except Exception as exc:
                        print(f"リファレンスの処理中に例外が発生しました: {exc}")

        # 各結果を統合して最終的なアウトプットとする
        combined_topics = "\n\n".join(expanded_topics_list)
        return dspy.Prediction(expanded_topics=combined_topics)
